In [1]:
import random
import time
from IPython.display import clear_output

class Card:
    '''
        Card object that consists of a suit, face, and value
    '''
    def __init__(self,suit,face):
        
        self._suit = suit
        
        #face is accepted as a dictionary and iterated through to assign 
        #the face and value respectively 
        for key,value in face.items():
            self.__face = key
            self.__value = value
    
    def getFace(self):
        return self.__face
    
    def getValue(self):
        return self.__value
    
    def __str__(self):
        return f"{self.__face} of {self._suit}"
        
class Deck:    
    '''
        Deck object conisting of 52 card objects
    '''
    def __init__(self):
        
        #Define suits and faces for Card objects when creating a Deck object
        __suits = ("Hearts","Diamonds","Clovers","Spades")
        __faces = {"Two":2,"Three":3,"Four":4,"Five":5,"Six":6,"Seven":7,"Eight":8,"Nine":9,
                "Ten":10,"Jack":10,"Queen":10,"King":10,"Ace":11}
        
        #initialize an empty cards list to store Card objects
        self.__cards = []
        
        #iterate through suites and faces, passing each combonation to 
        #Card class as a dictioary to create a card object and append to deck/cards list
        for key,value in __faces.items():
            for suit in __suits:
                card = Card(suit,{key:value})
                self.__cards.append(card)
                
    #Shuffle: shuffle all card objects in the deck
    def shuffle(self):
        random.shuffle(self.__cards)
        
    #dealOne: removes one Card object and returns it
    def dealOne(self):
        if len(self.__cards) > 0:           
            return self.__cards.pop()
        else:
            return None
        
    def __len__(self):
        return len(self.__cards)
        
class Hand:
    
    def __init__(self):
        self.__cards = []
        self.__tot = 0
        self.__aces = 0
        
    #acceptCard: receives a Card object and adds value to hand total
    def acceptCard(self,card):
        if card:
            self.__cards.append(card)
            self.total()
        
    #total: sums up all Card object values, special handling of Aces Card
    def total(self):
        self.__tot = 0
        self.__aces = 0
        
        for card in self.__cards:
            self.__tot += card.getValue()
            
            if card.getFace() == "Ace":
                self.__aces += 1
                
        while self.__tot > 21 and self.__aces > 0:
            self.__tot -= 10
            self.__aces -= 1
            
        return self.__tot
    
    #clearHand: resets hand object values
    def clearHand(self):
        self.__cards = []
        self.__tot = 0
        self.__aces = 0
            
    def getCards(self):
        return self.__cards
    
    def getTotal(self):
        return self.__tot
            
    def __str__(self):
        output = "Hand:\n"
        for card in self.__cards:
            output += f"  {card}\n"
            
        output += f"Card Sum: {self.__tot}"
        
        return output

class Player:
    
    def __init__(self,name,bankroll):
        self.__name = name
        self.__hand = Hand()
        self.__bankroll = float(bankroll)
        self.__bet = 0.0
        self.__standings = {"Wins":0,"Loses":0,"Busts":0}
        
    #placeBet: receives a bet amount and removes it from players bankroll
    def placeBet(self,bet):
        self.__bet = bet
        self.__bankroll -= bet
    
    def getName(self):
        return self.__name
    
    def getHand(self):
        return self.__hand
    
    def getBankroll(self):
        return self.__bankroll
    
    def setBankroll(self,amount):
        self.__bankroll = amount
    
    def getBet(self):
        return self.__bet
    
    def setBet(self,bet):
        self.__bet = bet
    
    def getStandings(self):
        return self.__standings
    
    def setStandings(self,standing,value):
        self.__standings[standing] = value
        
    def __str__(self):
        return f"\033[91m\033[1m{self.getName()}\033[0m - Bankroll ${self.getBankroll()} Bet ${self.getBet()}\n{str(self.getHand())}\n"
    
    def __format__(self,spec="l"):
        return self.__str__()
    
class AIPlayer(Player):
    
    #dealerMove: receievs players hand total to compare and decide to hit or stand, receives deck to hit from
    def dealerMove(self,playerHand,deck):
        while len(deck) > 0:
            if self.getHand().getTotal() == 21:
                break
            elif self.getHand().getTotal() > 21:
                break
            elif 21 > self.getHand().getTotal() > playerHand:
                break
            elif playerHand > 21 > self.getHand().getTotal():
                break
            elif self.getHand().getTotal() < 16:
                self.getHand().acceptCard(deck.dealOne())
            else:
                break

        return deck
    
    def __str__(self):
        return f"\033[1m{self.getName()}\033[0m\n{str(self.getHand())}\n"
    
    #special formatting to only display one card of dealers hand
    def __format__(self,spec):
        
        if spec == "s":
            return f"\033[1m{self.getName()}\033[0m\n{str(self.getHand().getCards()[0])}\n"
        
        return f"\033[1m{self.getName()}\033[0m\n{str(self.getHand())}\n"
    
def displayTable(players,reveal="s",message=""):
    '''
        Receives list of players, and messages needed to be displayed.
        Clears the screen and displays accordingly
    '''
    #any messages should be displayed at the top of table/screen
    canvas = message
    
    #iterate through player list and add player information
    for player in players:
        if reveal == "s":
            canvas += f"{player:s}"
        else:
            canvas += f"{player:l}"
        canvas += f"______________\n"
        
    #display player standings at the bottom of table/screen
    canvas += f"{players[1].getStandings()}\n"
    
    #clear the screen before displaying all content
    clear_output()
    
    #needed to add sleep time as clear was preventing input to display,
    #this is a known issue in jupyter
    time.sleep(.02)
    
    print(canvas)
    
def checkCards(player1,hand2):
    '''
        Receives player1 object and player2 hand total.
        Determines winner, adds and substracts to player1 bankroll accordingly,
        adds to player1 standings
    '''
    hand1 = player1.getHand().getTotal()
    bet = player1.getBet()
    player1.setBet(0)
    
    if hand1 == hand2 <= 21:
        player1.setBankroll(player1.getBankroll() + bet)
        return "Tie\n\n", player1
    elif hand1 == 21:
        player1.setBankroll(player1.getBankroll() + bet*3)
        player1.setStandings("Wins",player1.getStandings()["Wins"] + 1)
        return "BLACK JACK!!\n\n",player1
    elif hand2 == 21:
        player1.setStandings("Loses",player1.getStandings()["Loses"] + 1)
        return "House black jack!\n\n", player1
    elif hand1 > 21:
        player1.setStandings("Busts",player1.getStandings()["Busts"] + 1)
        player1.setStandings("Loses",player1.getStandings()["Loses"] + 1)
        return "BUST!\n\n", player1
    elif 21 >= hand1 > hand2 or hand2 > 21 >= hand1:
        player1.setBankroll(player1.getBankroll() + bet*2)
        player1.setStandings("Wins",player1.getStandings()["Wins"] + 1)
        return f"{player1.getName()} WINS!\n\n",player1
    elif 21 >= hand2 > hand1 or hand1 > 21 >= hand2:
        player1.setStandings("Loses",player1.getStandings()["Loses"] + 1)
        return "House WINS!\n\n",player1
    elif hand1 > 21 and hand2 > 21:
        return "No winners this round\n\n",player1
    else:
        return "lose\n\n",player1

In [2]:
#create ai, house has unlimited funds
player2 = AIPlayer("House",0) #ai
error = ""
rnd = 0
aDeck = Deck()
aDeck.shuffle()

print("Welcome to blackjack!")
name = input("What's your name? ")

while True:
    try:
        bankroll = float(input("How much money are you bringing to the table? $"))
        if bankroll < 5:
            raise ValueError()
    except ValueError:
        print("\033[91mThis isn't a high stakes table, but please enter an amount greater than $5.\033[0m")    
    except:
        print("\033[91mStop messing around.\033[0m")
    else:
        player1 = Player(name,bankroll)
        break
        
while True:
    rnd += 1
    bet = 0.00
    spec = "s"
    
    while len(aDeck) > 4:                   
        
        try:
            clear_output()
            player1.getHand().clearHand();
            player2.getHand().clearHand();
            
            print(f"{error}Bankroll ${player1.getBankroll()}")
            bet = float(input("Place your bets $"))
            
            if bet > player1.getBankroll():
                error = "Insificient funds compradre..\n"
                continue
            elif bet <= 0:
                error = "That's pocket lint. Please enter a valid amount"
            else:
                player1.placeBet(bet)
                player1.getHand().acceptCard(aDeck.dealOne())
                player2.getHand().acceptCard(aDeck.dealOne())
                player1.getHand().acceptCard(aDeck.dealOne())
                player2.getHand().acceptCard(aDeck.dealOne())
                error = ""
                break
        except ValueError:
            error = f"Please enter a valid amount [1-{player1.getBankroll()}]\n"
            continue
        except TypeError:
            error = f"Try again\n"
            continue
        except:
            break
    else:
        print("No more cards, good game")
        break
    
    while len(aDeck) > 0:
        
        try:
            if player1.getHand().getTotal() == 21:
                break
            elif player2.getHand().getTotal() == 21:
                spec = "l"

            move = ""
            displayTable([player2,player1],spec,error)
            error = ""
            move = int(input("Move [1-Hit;2-Stand]: "))
            
            if move == 1:
                player1.getHand().acceptCard(aDeck.dealOne())
                if player1.getHand().getTotal() == 21:
                    break
                elif player1.getHand().getTotal() > 21:
                    break
            else:
                break
        except:
            error += "\033[91m-Please enter 1 or 2.-\033[0m\n"
            continue
                
    else:
        print("No more cards to deal")
        
    aDeck = player2.dealerMove(player1.getHand().getTotal(),aDeck)   

    message,player1 = checkCards(player1,player2.getHand().getTotal())
    
    displayTable([player2,player1],"l",message)    
    
    if player1.getBankroll() == 0:
        print("Seems you're out of funds, better luck next time")
        break
        
    again = input(f"Ready for round {rnd+1}[y,yes/n,no]? ")
    if again.upper() in ("Y","YES"):
        pass
    else:
        break


sekthree WINS!

House
Hand:
  Jack of Hearts
  Two of Hearts
  Four of Diamonds
Card Sum: 16
______________
sekthree - Bankroll $180.0 Bet $0
Hand:
  Seven of Diamonds
  Ten of Diamonds
Card Sum: 17
______________
{'Wins': 6, 'Loses': 4, 'Busts': 0}

Ready for round 11[y,yes/n,no]? y
No more cards, good game
